In [ ]:
import json
import psycopg2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Database configuration
db_config = json.load(open('config.json'))

# Query to fetch data
query = """
SELECT 
    P.article_number,
    P.category,
    P.department,
    DC.d0_inventory,
    DC.d1_inventory,
    DC.d2_inventory,
    DC.d3_inventory,
    DC.d4_inventory,
    DC.d5_inventory,
    DC.d6_inventory
FROM 
    public.dailycheckin DC
JOIN 
    public.products P
ON 
    P.id = DC.product_id
ORDER BY 
    P.article_number, DC.year, DC.week;
"""

# Category Prediction Model
class CategoryPredictionModel(nn.Module):
    def __init__(self, num_categories, num_departments, embed_dim=8):
        super(CategoryPredictionModel, self).__init__()
        # Embedding layer for department
        self.department_embedding = nn.Embedding(num_departments, embed_dim)
        
        # Fully connected layers
        self.fc1 = nn.Linear(embed_dim + 7, 64)  # Embedding + 7 inventory features
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_categories)  # Output: number of categories (classes)

    def forward(self, inventory_features, department_ids):
        # Embedding lookup
        dept_emb = self.department_embedding(department_ids)
        
        # Combine embeddings and inventory features
        combined = torch.cat((inventory_features, dept_emb), dim=1)
        
        # Fully connected layers
        x = torch.relu(self.fc1(combined))
        x = torch.relu(self.fc2(x))
        output = self.fc3(x)  # Output logits for classification
        return output


In [ ]:
def fetch_data_from_db():
    conn = psycopg2.connect(**db_config)
    df = pd.read_sql(query, conn)
    conn.close()
    return df

def prepare_category_data(df):
    # Encode category as labels
    category_encoder = LabelEncoder()
    df['category_encoded'] = category_encoder.fit_transform(df['category'])

    # Map department to indices
    department_mapping = {dept: idx for idx, dept in enumerate(df['department'].unique())}
    df['department_id'] = df['department'].map(department_mapping)

    # Prepare tensors
    inventory_features = torch.tensor(df[['d0_inventory', 'd1_inventory', 'd2_inventory', 
                                           'd3_inventory', 'd4_inventory', 'd5_inventory', 
                                           'd6_inventory']].fillna(0).values, dtype=torch.float)
    department_ids = torch.tensor(df['department_id'].values, dtype=torch.long)
    category_labels = torch.tensor(df['category_encoded'].values, dtype=torch.long)

    return inventory_features, department_ids, category_labels, len(department_mapping), len(category_encoder.classes_), category_encoder

def train_category_prediction_model():
    print("Fetching data...")
    df = fetch_data_from_db()

    print("Preparing data...")
    inventory_features, department_ids, category_labels, num_departments, num_categories, category_encoder = prepare_category_data(df)

    # Train-test split
    X_train, X_test, dept_train, dept_test, y_train, y_test = train_test_split(
        inventory_features, department_ids, category_labels, test_size=0.2, random_state=42
    )

    # Initialize model, loss, optimizer
    model = CategoryPredictionModel(num_categories, num_departments)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    epochs = 10
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X_train, dept_train)
        loss = criterion(outputs, y_train)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

    # Testing loop
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test, dept_test)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = (predicted == y_test).float().mean().item()
        print(f"Test Accuracy: {accuracy * 100:.2f}%")

    return model, category_encoder

def guess_category(model, category_encoder, inventory_pattern, department):
    # Convert inputs to tensors
    inventory_tensor = torch.tensor([inventory_pattern], dtype=torch.float)
    department_tensor = torch.tensor([department], dtype=torch.long)

    # Predict category
    model.eval()
    with torch.no_grad():
        logits = model(inventory_tensor, department_tensor)
        _, predicted_class = torch.max(logits, 1)

    # Decode category
    predicted_category = category_encoder.inverse_transform(predicted_class.numpy())
    return predicted_category[0]

if __name__ == "__main__":
    model, category_encoder = train_category_prediction_model()

    # Example: Guess category
    inventory_pattern = [10, 15, 20, 25, 20, 15, 10]  # Example pattern
    department = 0  # Example department index
    category = guess_category(model, category_encoder, inventory_pattern, department)
    print(f"Predicted Category: {category}")
